# Used Libraries

In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from typing import Optional
from pydantic import BaseModel


from fastapi.encoders import jsonable_encoder
from fastapi import FastAPI
from enum import Enum
import os
import io
import pickle
import uvicorn
import nest_asyncio

# Load Models

These are pre-trained models weights on Covid dataset, you can find more of this on link below:

[Arabic-Hands-on-Analysis-Clustering-and-Classification-of-Large-Arabic-Twitter-Data-set-on-COVID19](https://github.com/Abdelrahmanrezk/Arabic-Hands-on-Analysis-Clustering-and-Classification-of-Large-Arabic-Twitter-Data-set-on-COVID19)

In [3]:
log_reg_model = pickle.load(open("logistic_regression_93tr_91ts.sav", "rb"))
randforest_model = pickle.load(open("RandomForestClassifier_99tr_92_ts.sav", "rb"))
tf_idf_model = pickle.load(open("tf_idf_vectorizer_50000_fetures.pickle", "rb"))

# Load Data To test on

In [4]:
covid_tweets_data = pd.read_csv('test_data.csv')
covid_tweets_data.head()

,cleaned_tweet_text,class
0,ممثل منظمه الصحه العالميه في مصر يحدث في مصر ا...,1
1,وباء وانتشر,1
2,هجوم عنيف المستشار اسامه ابوالمجد علي مصلحه ال...,1
3,طيب وانتم بالله عندكم قوه نفس امريكا او الصين ...,0
4,بحر اليابان وليس بحر الصين,0


In [5]:
def classify_tweet(tweets, model):
    '''
    The function used to classify tweets, but during to the model design it take a list of tweets,
    and return numpy array, so we have do some manipulation before return the data.
    
    Argument:
        tweets: dictionary of tweet text and default value = 0
        model: Enum of two string which the model you need to choose
    Return:
        classifed_tweet: dictionary of tweet text and default value = negative
    '''
    
    # get the tweet text as string then convert to list
    tweet = [tweets['tweet_text']]
    
    # Get the text features using Tf-Idf vectorization model
    tweets_features = tf_idf_model.transform(tweet)
    
    # convert to sparse matrix instead of compressed space type
    tweets_features_array = tweets_features.toarray()
    
    
    # Check which model you need to predicit 
    if model == "Logistic_Model":
        print("Prediction using Logistic Model")
        print("="*50)
        predict = log_reg_model.predict(tweets_features_array)
    else:
        print("Prediction using RandomForest Model")
        print("="*50)
        predict = randforest_model.predict(tweets_features_array)
        
    
    pred_result = 'Positive' if predict[0] == 1 else 'Negative'
    
    # Return dictionary
    classifed_tweet = {
        'tweet_text': tweet,
        'predict_class': pred_result
#         'predict_class': predict
    }
    return classifed_tweet

In [6]:
def test_tweet(tweet_indx_with_100, model_type):
    test_tweet = list(covid_tweets_data['cleaned_tweet_text'])[tweet_indx_with_100]
    print(test_tweet)
    print("=============== True Value for this tweet ===================")
    print("=============== " + str(covid_tweets_data['class'][tweet_indx_with_100]) + " ===================")
    
    print("="*50)
    tweet_dict = {
        'tweet_text': test_tweet,
        'predict_class': '0'
    }
    
    return classify_tweet(tweet_dict, model_type)

In [7]:
test_tweet(0, "Logistic_Model")

ممثل منظمه الصحه العالميه في مصر يحدث في مصر اثق تماما في الاجراءات تتخذها مصر لمواجهه وباء محتمل MBCMASR 
=============== True Value for this tweet ===================
=============== 1 ===================
Prediction using Logistic Model


{'tweet_text': ['ممثل منظمه الصحه العالميه في مصر يحدث في مصر اثق تماما في الاجراءات تتخذها مصر لمواجهه وباء محتمل MBCMASR '],
 'predict_class': 'Positive'}

In [8]:
test_tweet(3, "")

طيب وانتم بالله عندكم قوه نفس امريكا او الصين او رسيا مثلا اجب 
=============== True Value for this tweet ===================
=============== 0 ===================
Prediction using RandomForest Model


{'tweet_text': ['طيب وانتم بالله عندكم قوه نفس امريكا او الصين او رسيا مثلا اجب '],
 'predict_class': 'Negative'}

In [9]:
# Assign an instance of the FastAPI class to the variable "app".
# You will interact with your api using this instance.
app = FastAPI(title='Deploying a ML Model with FastAPI')

# List available models using Enum
class Model(str, Enum):
    Logistic_Model = "Logistic_Model"
    Rnd_Forest_Model = "Random_Forest_Model"


class Tweet_Text(BaseModel):
    tweet_text: str = """ممثل منظمه الصحه العالميه في مصر يحدث في مصر اثق تماما في الاجراءات تتخذها مصر لمواجهه وباء محتمل MBCMASR """
    predict_class: Optional[str] = 'Negative'
        

@app.get("/")
def home():
    # Once you go to this link you will see the get and post method below to trying out
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:5000/docs."


# This endpoint handles all the logic necessary for the object detection to work.
# It requires the desired model and the dictionary of tweet and default class as we give default values to us
# In the api you can try other tweet from some_tweets below
@app.post("/predict") 
def prediction(model: Model, tweet: Tweet_Text):
    
    # Encode the retrived request data 
    tweet = jsonable_encoder(tweet)
    
    # Run our model
    classifed_tweet = classify_tweet(tweet, model=model)
    
    return classifed_tweet

In [10]:
some_tweets = list(covid_tweets_data['cleaned_tweet_text'])[:10]
some_tweets # choose any tweet to test with the api

['ممثل منظمه الصحه العالميه في مصر يحدث في مصر اثق تماما في الاجراءات تتخذها مصر لمواجهه وباء محتمل MBCMASR ',
 'وباء وانتشر ',
 'هجوم عنيف المستشار اسامه ابوالمجد علي مصلحه الجمارك ومكاتب التمثيل التجاري وجهاز حمايه المنافسه لتسببهم في القضاء علي الاستيراد الموازي والمستوردين ماتسبب في تفشي ظاهره الاحتكار ',
 'طيب وانتم بالله عندكم قوه نفس امريكا او الصين او رسيا مثلا اجب ',
 'بحر اليابان وليس بحر الصين ',
 'مانحبلك الخساره اشتري كمامات بس ',
 'نفس الكلام بيحصل عندنا في السوداان الله بيكون دا وباء ولا شنو الحاصل ',
 'اما ووهان رو به فاصله سال دیدم به حدی رشدش سریع بود که باورکردنی نبود یعنی اصلا در مخیله ایرانی نمی كنجید که چطور در عرض سال یک شهر تا این حد می تونه تغییر کنه ',
 'المطلوب طيب خطتين وباء الخنق بس للتهديد مو انك تقتليه ',
 'وباء معدي البلد اليومين دول اللهم احفظنا ']

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# uvicorn is fast Asynchronous Server Gateway Interface (ASGI) uvicorn handles the serving
# Spin up the server!    
uvicorn.run(app, host=host, port=5000)

INFO:     Started server process [9340]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:45004 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:45004 - "GET /openapi.json HTTP/1.1" 200 OK
Prediction using Logistic Model
INFO:     127.0.0.1:45008 - "POST /predict?model=Logistic_Model HTTP/1.1" 200 OK
Prediction using RandomForest Model
INFO:     127.0.0.1:45016 - "POST /predict?model=Random_Forest_Model HTTP/1.1" 200 OK
Prediction using RandomForest Model
INFO:     127.0.0.1:45020 - "POST /predict?model=Random_Forest_Model HTTP/1.1" 200 OK
Prediction using Logistic Model
INFO:     127.0.0.1:45024 - "POST /predict?model=Logistic_Model HTTP/1.1" 200 OK
Prediction using Logistic Model
INFO:     127.0.0.1:45028 - "POST /predict?model=Logistic_Model HTTP/1.1" 200 OK
Prediction using RandomForest Model
INFO:     127.0.0.1:45032 - "POST /predict?model=Random_Forest_Model HTTP/1.1" 200 OK
